# PHASE 3: ???
- TODO: fill in with Prof's description

# PLANNING

DATA PREP
- stratified split 

ML METHODS
- Binary classification
- Multiclass classification
- KNN?

EVALUATION
- Confusion Matrix: TP, FP, TN, FN, FP
  - Then F1 and stuff...????

SOURCE
- https://www.learndatasci.com/glossary/binary-classification/ 

# IMPORTS

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import sklearn